## Setting

In [1]:
import os

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

# maic_dir = '/mnt/backup/'
# base_dir = os.path.join(maic_dir, 'working')
base_dir = os.getcwd()
input_dir = '/mnt/NM/dataset/'
temp_dir = os.path.join(base_dir, 'convert_data')

# maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(base_dir, 'models/module')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///mnt/submission/submit_files/nnUNet
  Attempting uninstall: nnunet
    Found existing installation: nnunet 1.7.0
    Can't uninstall 'nnunet'. No files were found to uninstall.
  Running setup.py develop for nnunet
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [10]:
import shutil
import json
import numpy as np
import pandas as pd

from collections import OrderedDict
from scipy import special
import copy
import cv2
from tqdm import tqdm
import h5py

# must install
import pydicom
import nibabel as nib
import SimpleITK as sitk

In [6]:
task_name = 'Task55_PETCT'
convert_name = 'Task555_PETCT'

main_dir = os.path.join(base_dir, 'nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')
Prediction_dir = os.path.join(main_dir, 'nnUNet_Prediction_Results')
result_dir = os.path.join(Prediction_dir, convert_name)

staple_dir = os.path.join(Prediction_dir, 'staple')

# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)

# 3. Directory
maybe_mkdir_p(result_dir)
maybe_mkdir_p(staple_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = './models' #os.path.join(main_dir, 'nnUNet_trained_models')

print('Setting Completed!')

Setting Completed!


## Testdataset Preprocessing

In [7]:
def hdf2nifti(hdf_folder: str, save_folder: str):
    # hdf_folder : [train_dir, test_dir] hdf5 file path
    # save_folder : [imagesTr, imagesTs] Save Folder path
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTs'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTr'))
    print('Creating "{}" Image & Label ..'.format(os.path.basename(os.path.normpath(save_folder))))
    hdf5_files = os.listdir(hdf_folder)


    for hdf5_file in hdf5_files:


        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        f_i.close()

        SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT = ctarr.shape
        images = np.empty([SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT, 0], dtype=np.single)

        image_ct = np.expand_dims(ctarr, axis=3)
        images = np.append(images, image_ct, axis=3)
        image_pet = np.expand_dims(petarr, axis=3)
        images = np.append(images, image_pet, axis=3)


        hdf5_file_NAME = hdf5_file

        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, 'imagesTs/{}.nii.gz'.format(hdf5_file[:-8])))


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(save_folder))))
    print('Image Patient : {}'.format(len(os.listdir(input_dir))))

hdf2nifti(input_dir, task_dir)

Creating "Task55_PETCT" Image & Label ..


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/NM/dataset/'

In [19]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)

    overwrite_json_file = True
    json_file_exist = False

    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist == False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = "PETCT"
        json_dict['description'] = "Medical Image AI Challenge 2021"
        json_dict['tensorImageSize'] = "4D"
        json_dict['reference'] = "https://maic.or.kr/competitions/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "18/10/2021"

        json_dict['modality'] = {
            "0": "CT",
            "1": "PET"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "Aorta"
        }

        train_ids = sorted(os.listdir(imagesTr))
        test_ids = sorted(os.listdir(imagesTs))
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] #(i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')
json_mk(task_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/temp/nnUNet/nnunet/Tasks/Task55_PETCT'

In [ ]:

!nnUNet_convert_decathlon_task -i ./convert_data/nnUNet/nnunet/Tasks/Task55_PETCT -output_task_id 555 # -i : task_dir
# !nnUNet_plan_and_preprocess -t 555 # --verify_dataset_integrity


## File path

## Prediction

In [13]:
!nnUNet_predict -i ./convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs -o ./result -t 555 -tr nnUNetTrainerV2 -m 2d --disable_tta



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  ./model/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 1 unique case ids, here are some examples: ['23010018_20141226']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 1
number of cases that still need to be predicted: 1
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['./model/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0', './model/nn

## Submission

In [ ]:
def get_suv_params(ptarr, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    suvmax = np.max(ptarr*roi)
    suvmean = np.sum(ptarr*roi)/np.sum(roi)
    return suvmax, suvmean

def get_vol_params(ptzoom, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    return np.prod(ptzoom) * np.sum(roi)

In [ ]:
PN_list = os.listdir('./result')

for pn_l in PN_list:
    _, ext = os.path.splitext(pn_l)
    if ext == '.gz':
        
        pn = pn_l[:-7]

        ptarr = np.array(nib.load('./convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0001.nii.gz'.format(pn)).dataobj)
        ctarr = np.array(nib.load('./convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0000.nii.gz'.format(pn)).dataobj)
        pred_arr = np.array(nib.load('./result/{}.nii.gz'.format(pn)).dataobj)

        size = ([4.07283, 4.07283, 3.])

        #Calculate Mean SUV and Max SUV

        suvmax, suvmean = get_suv_params(ptarr, pred_arr)

        #Calculate Volume

        aorvol = get_vol_params(size, pred_arr)
        
        PATIENT_NUM = pn

        data = {'case' : [PATIENT_NUM], 'PD_Aorta_volume' : [aorvol], 'PD_SUVmean' : [suvmean]}
        df = pd.DataFrame(data)

        # .to_csv 
        if not os.path.exists('submission.csv'):
            df.to_csv('submission.csv', index=False, mode='w')
        else:
            df.to_csv('submission.csv', index=False, mode='a', header=False)


In [71]:
num_list = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
print(num_list)
numm = np.uint8((num_list >= 0.3))
print(numm)

[[0.1 0.2 0.3]
 [0.4 0.5 0.6]]
[[0 0 1]
 [1 1 1]]


In [32]:
!htop

> 1 root       20   0  6004  3728  3248 S  0.0  0.0  0:00.02 /bin/bash /root/e   0  547M 51520 14416 S  0.7  0.0  0:01.81 /usr/bin/python3h /root/e14 root20   0  293M  107M 18808 S  0.0  0.1  0:00.00 /usr/bin/python3 15 root20   0  293M  107M 18808 S  0.0  0.1  0:00.07 /usr/bin/python3 8 root20   0  293M  107M 18808 S  0.0  0.1  1:27.03 /usr/bin/python320 root20   0  547M 51520 14416 S  0.0  0.0  0:00.00 /usr/bin/python3 F1Help  F2Setup F3SearchF4FilterF5Tree  F6SortByF7Nice -F8Nice +F9Kill  F10Quit[       0.0%]6  [       0.0%]   18 [       0.0%]   30 [       0.0%]   42 [       0.0%]7  [       0.0%]   19 [       0.0%]   31 [       0.0%]   43 [       0.0%]8  [       0.0%]   20 [       0.0%]   32 [       0.0%]   44 [       0.0%]9  [       0.0%]   21 [       0.0%]   33 [       0.0%]   45 [       0.0%]10 [       0.0%]   22 [       0.0%]   34 [       0.0%]   46 [       0.0%]11 [       0.0%]   23 [       0.0%]   35 [       0.0%]   47 [       0.0%]12 [       0.0%]   24 [       0.0%]   36 [  

In [1]:
! nvidia-smi

Mon Nov 15 06:38:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1C.0 Off |                    0 |
| N/A   28C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
model_dir = '/mnt/backup/temp_submission/models/'
model_1 = os.path.join(model_dir, 'nnUNet/2d/')
# fold_list = ['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4']
# fold_list = [fold_0, fold_1, fold_2, fold_3, fold_4]

Plan = 'Task555_PETCT/nnUNetTrainerV2_Loss_Dice__nnUNetPlansv2.1/'

fold_all = os.path.join(model_1, Plan, 'all/')
fold_0 = os.path.join(model_1, Plan, 'fold_0/')
fold_1 = os.path.join(model_1, Plan, 'fold_1/')
fold_2 = os.path.join(model_1, Plan, 'fold_2/')
fold_3 = os.path.join(model_1, Plan, 'fold_3/')
fold_4 = os.path.join(model_1, Plan, 'fold_4/')

fold_list = [fold_0, fold_1, fold_2, fold_3, fold_4]

In [7]:
for fold in fold_list:
    if os.path.exists(fold):
        shutil.rmtree(fold)
        shutil.copytree(fold_all, fold)
    else:
        shutil.copytree(fold_all, fold)

In [33]:
print(len(os.listdir('/mnt/dataset')))
print(os.listdir('/mnt'))
print(os.listdir('/mnt/dataset'))

80
['dataset']
['23090580_20131226.hdf5', '23090618_20161212.hdf5', '23090628_20150204.hdf5', '23090643_20121227.hdf5', '23090644_20131216.hdf5', '23090636_20121018.hdf5', '23090585_20130213.hdf5', '23090572_20130226.hdf5', '23090603_20141212.hdf5', '23090601_20130225.hdf5', '23090569_20120607.hdf5', '23090599_20140701.hdf5', '23090597_20130227.hdf5', '23090627_20160608.hdf5', '23090559_20150812.hdf5', '23090614_20120402.hdf5', '23090557_20130717.hdf5', '23090634_20150409.hdf5', '23090581_20130626.hdf5', '23090626_20160119.hdf5', '23090568_20121018.hdf5', '23090623_20120406.hdf5', '23090615_20140403.hdf5', '23090640_20140711.hdf5', '23090613_20130208.hdf5', '23090641_20160510.hdf5', '23090609_20120510.hdf5', '23090560_20160114.hdf5', '23090563_20151216.hdf5', '23090596_20150112.hdf5', '23090632_20130807.hdf5', '23090598_20130103.hdf5', '23090584_20120523.hdf5', '23090633_20120403.hdf5', '23090607_20120420.hdf5', '23090594_20160706.hdf5', '23090622_20150105.hdf5', '23090583_20160308.hdf

In [30]:
import os
import shutil
# Copy model..
# shutil.copytree('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/3d_fullres/Task555_PETCT/nnUNetTrainerV2_Loss_Dice__nnUNetPlansv2.1/', '/tf/backup/temp_submission/models/nnUNet/3d_fullres/Task555_PETCT/nnUNetTrainerV2_Loss_Dice__nnUNetPlansv2.1/')
# shutil.copytree('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/3d_fullres/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/', '/tf/sub_temp/models/nnUNet/3d_fullres/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1')
# shutil.copytree('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2_Loss_CEGDL__nnUNetPlansv2.1/', '/tf/sub_temp/models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2_Loss_CEGDL__nnUNetPlansv2.1/')
# shutil.copytree('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2_Loss_DiceTopK10__nnUNetPlansv2.1/', '/tf/sub_temp/models/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2_Loss_DiceTopK10__nnUNetPlansv2.1/')
# shutil.copytree('/tf/backup/temp_submission/models/module/', '/tf/sub_temp/models/module')
# shutil.copytree('/tf/submission/submitted/21-10-27_13:29:02-models/', '/tf/temp_submission/models')
# shutil.copytree('/tf/backup/nnUNet', '/tf/temp_submission/models/module')
# shutil.copytree('/tf/backup/temp_submission/', '/tf/sub_temp/')
# shutil.copytree('/tf/temp_submission/', '/tf/backup/temp_submission')
# shutil.copytree('/tf/sub_temp/', '/tf/sub_temp_temp/')
# shutil.copytree('/tf/submission/submit_files', /tf/sub_temp_temp/')


In [19]:
# shutil.rmtree('/tf/submission/submit_files/models')

In [2]:
# shutil.rmtree('/tf/temp/models/nnUNet/')

In [8]:
# shutil.rmtree('/tf/backup/temp_submission/models/nnUNet/2d/')

In [26]:
# shutil.rmtree('/tf/backup/working/nnUNet/nnunet/nnUNet_trained_models/nnUNet/3d_fullres/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/')

In [32]:
# os.listdir('/mnt/dataset')